In [ ]:
#Imports
from flask import Flask, render_template, request, jsonify
import nltk
import datetime
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
import random
import json
import pickle

stemmer = LancasterStemmer()
seat_count = 50

with open("intents.json") as file:
	data = json.load(file)
with open("data.pickle","rb") as f:
	words, labels, training, output = pickle.load(f)

#Function to process input
def bag_of_words(s, words):
	bag = [0 for _ in range(len(words))]
	
	s_words = nltk.word_tokenize(s)
	s_words = [stemmer.stem(word.lower()) for word in s_words]

	for se in s_words:
		for i,w in enumerate(words):
			if w == se:
				bag[i] = 1

	return np.array(bag)

from tensorflow.python.framework import ops

ops.reset_default_graph()
net = tflearn.input_data(shape = [None, len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[0]), activation = "softmax")
net = tflearn.regression(net)

#Loading existing model from disk
model = tflearn.DNN(net)
model.load("model.tflearn")

miss={"tag" : "missing",
     "responsee" : "type something"}

app = Flask(__name__)

@app.route('/')
def index():
	return render_template('index.html')

@app.route('/get')
def get_bot_response():
    global seat_count
    message = request.args.get('msg')
    
    if message:
        message = message.lower()
        message=message.replace('+',' ')
        results = model.predict([bag_of_words(message,words)])[0]
        result_index = np.argmax(results)
        tag = labels[result_index]
        print("result : ",tag)  
        print(message)

        if results[result_index] > 0.5:
            if tag == "checkstock":
                
                response = "Your stock is follows"
                data2 = {"tag" : tag,
                         "responsee" : str(response),}
        
                return jsonify(data2)
                

            elif tag == "available_tables":
                response = "There are " + str(seat_count) + " tables available at the moment."

           
            else:
                for tg in data['intents']:
                    if tg['tag'] == tag:
                        responses = tg['responses']
                response = random.choice(responses)
                data2 = {"tag" : tag,
                         "responsee" : str(response),}
                return jsonify(data2)
        else:
            response = "I didn't quite get that, please try again."
            data2 = {
            "tag" : tag,
            "responsee" : str(response),
            }
        return jsonify(data2)
    return miss


    
if __name__ == "__main__":
    app.run()

INFO:tensorflow:Restoring parameters from C:\Users\suriy\apii\app\app\model.tflearn
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Aug/2022 16:09:21] "GET /get?msg=sddnwd%20wd%20wd%20 HTTP/1.1" 200 -


result :  greeting
sddnwd wd wd 


127.0.0.1 - - [13/Aug/2022 16:10:13] "GET /get?msg= HTTP/1.1" 200 -
127.0.0.1 - - [13/Aug/2022 16:10:15] "GET /get?msg= HTTP/1.1" 200 -
127.0.0.1 - - [13/Aug/2022 16:10:52] "GET /get?msg=hello HTTP/1.1" 200 -


result :  greeting
hello


...

result :  greeting
hello


127.0.0.1 - - [13/Aug/2022 16:11:01] "GET /get?msg=hello HTTP/1.1" 200 -


result :  greeting
hello


127.0.0.1 - - [13/Aug/2022 16:11:05] "GET /get?msg=hello HTTP/1.1" 200 -


result :  greeting
hello
